<a href="https://colab.research.google.com/github/solllull/entregasML/blob/main/tp2_experimentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TP 2: ENTRENAMIENTO Y EVALUACIÓN DE MODELOS**

In [ ]:
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import minmax_scale
from datetime import datetime
from matplotlib import gridspec
import plotly.express as px
from matplotlib.lines import Line2D

from collections import defaultdict
from matplotlib import pyplot as plt
import seaborn as sns

from IPython.display import HTML, display
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import train_test_split

from sklearn_pandas import DataFrameMapper
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelBinarizer
from sklearn.ensemble import RandomForestClassifier
import graphviz  
from sklearn.tree import export_graphviz

In [ ]:
dataset = pd.read_csv('weatherAUS.csv')
dataset.shape

In [ ]:
# eliminación de filas con valores nulos
dataset = dataset.dropna(how='any',axis=0)
# convertir columna Date a tipo datetime
dataset['Date'] = pd.to_datetime(dataset['Date'])

**ELECCIÓN DE MÉTRICA**

Decidimos utilizar **Accuracy** como métrica para evaluar los modelos ya que predecir si va a llover o no al día siguiente no es algo crítico. Además queremos saber en qué porcentaje de casos, el modelo que entrenamos acertó, para lo cual esta métrica resulta adecuada para interpretarlo.

# **FEATURE ENGINEERING**


**Agregado de columna "Season"**

In [ ]:
# agregar columna "Season" (estación)
conditions_seasons = [
        (dataset['Date'].dt.month.isin([1, 2, 3])), # summer
        (dataset['Date'].dt.month.isin([4, 5, 6])), # autumn
        (dataset['Date'].dt.month.isin([7, 8, 9])), # winter
        (dataset['Date'].dt.month.isin([10, 11, 12])) # spring
]

seasons = ['summer', 'autumn', 'winter', 'spring']

# nueva columna Season
dataset['Season'] = np.select(conditions_seasons, seasons)

# cambiar season donde corresponda
dataset.Season.mask((dataset['Date'].dt.month == 12) & (dataset['Date'].dt.day > 20), 'summer', inplace=True)
dataset.Season.mask((dataset['Date'].dt.month == 3) & (dataset['Date'].dt.day > 20), 'autumn', inplace=True)
dataset.Season.mask((dataset['Date'].dt.month == 6) & (dataset['Date'].dt.day > 20), 'winter', inplace=True)
dataset.Season.mask((dataset['Date'].dt.month == 9) & (dataset['Date'].dt.day > 20), 'spring', inplace=True)

# Gráficos
fig2 = plt.figure(constrained_layout=True, figsize = (30,30))
spec2 = gridspec.GridSpec(ncols=6, nrows=7, figure=fig2)
f2_ax1 = fig2.add_subplot(spec2[0, 0])
f2_ax2 = fig2.add_subplot(spec2[0, 1])
f2_ax3 = fig2.add_subplot(spec2[0, 2])
f2_ax4 = fig2.add_subplot(spec2[0, 3])
dataset[dataset.Season == 'summer'].RainTomorrow.value_counts().sort_index().plot.pie(autopct='%1.0f%%', ax=f2_ax1, title = 'summer',figsize=(15,15))
dataset[dataset.Season == 'spring'].RainTomorrow.value_counts().sort_index().plot.pie(autopct='%1.0f%%', ax=f2_ax2,title = 'spring',figsize=(15,15))
dataset[dataset.Season == 'autumn'].RainTomorrow.value_counts().sort_index().plot.pie(autopct='%1.0f%%', ax=f2_ax3,title = 'autumn',figsize=(15,15))
dataset[dataset.Season == 'winter'].RainTomorrow.value_counts().sort_index().plot.pie(autopct='%1.0f%%', ax=f2_ax4,title = 'winter',figsize=(15,15))

**Redondeo de valores numéricos**: sacamos precisión a los valores de columnas numéricas, redondeando a dos decimales.

In [ ]:
tmp = dataset.select_dtypes(include=[np.number])
dataset.loc[:, tmp.columns] = np.round(tmp, 2)

**División de columna "Date" en 3 nuevas columnas: Día, Mes y Año**

In [ ]:
# agregar columnas año, mes y día
dataset['Year'] = dataset['Date'].dt.year
dataset['Month'] = dataset['Date'].dt.month
dataset['Day'] = dataset['Date'].dt.day

dataset[['Date', 'Year', 'Month', 'Day']].sample(10)

**Agregado de columna "Region"**

In [ ]:
conditions_regions = [
        (dataset['Location'].isin(['Perth', 'PerthAirport'])), # western
        (dataset['Location'].isin(['AliceSprings', 'Darwin'])), # northen
        (dataset['Location'].isin(['Woomera', 'Nuriootpa', 'MountGambier'])), # south
        (dataset['Location'].isin(['Mildura','Sale','WaggaWagga','Canberra','Sydney','SydneyAirport','Portland','Williamtown','CoffsHarbour','Moree','Cobar'])), # newsouth
        (dataset['Location'].isin(['Cairns', 'Townsville', 'Brisbane'])), # queensland
        (dataset['Location'].isin(['Watsonia','Melbourne','MelbourneAirport'])), # victoria
        (dataset['Location'].isin(['Hobart'])), # tasmania
        (dataset['Location'].isin(['NorfolkIsland'])) #norfolk
]

regions = ['western', 'northen', 'south', 'newsouth', 'queensland', 'victoria', 'tasmania', 'norfolk']

# nueva columna Region
dataset['Region'] = np.select(conditions_regions, regions)

#estos gráficos no hacen falta, los hice para ver qué onda nomás
fig2 = plt.figure(constrained_layout=True, figsize = (30,30))
spec2 = gridspec.GridSpec(ncols=6, nrows=7, figure=fig2)
f2_ax1 = fig2.add_subplot(spec2[0, 0])
f2_ax2 = fig2.add_subplot(spec2[0, 1])
f2_ax3 = fig2.add_subplot(spec2[0, 2])
f2_ax4 = fig2.add_subplot(spec2[0, 3])
f2_ax5 = fig2.add_subplot(spec2[1, 0])
f2_ax6 = fig2.add_subplot(spec2[1, 1])
f2_ax7 = fig2.add_subplot(spec2[1, 2])
f2_ax8 = fig2.add_subplot(spec2[1, 3])
dataset[dataset.Region == 'western'].RainTomorrow.value_counts().sort_index().plot.pie(autopct='%1.0f%%', ax=f2_ax1, title = 'western',figsize=(15,15))
dataset[dataset.Region == 'northen'].RainTomorrow.value_counts().sort_index().plot.pie(autopct='%1.0f%%', ax=f2_ax2,title = 'northen',figsize=(15,15))
dataset[dataset.Region == 'south'].RainTomorrow.value_counts().sort_index().plot.pie(autopct='%1.0f%%', ax=f2_ax3,title = 'south',figsize=(15,15))
dataset[dataset.Region == 'newsouth'].RainTomorrow.value_counts().sort_index().plot.pie(autopct='%1.0f%%', ax=f2_ax4,title = 'newsouth',figsize=(15,15))
dataset[dataset.Region == 'queensland'].RainTomorrow.value_counts().sort_index().plot.pie(autopct='%1.0f%%', ax=f2_ax5,title = 'queensland',figsize=(15,15))
dataset[dataset.Region == 'victoria'].RainTomorrow.value_counts().sort_index().plot.pie(autopct='%1.0f%%', ax=f2_ax6,title = 'victoria',figsize=(15,15))
dataset[dataset.Region == 'tasmania'].RainTomorrow.value_counts().sort_index().plot.pie(autopct='%1.0f%%', ax=f2_ax7,title = 'tasmania',figsize=(15,15))
dataset[dataset.Region == 'norfolk'].RainTomorrow.value_counts().sort_index().plot.pie(autopct='%1.0f%%', ax=f2_ax8,title = 'norfolk',figsize=(15,15))

**Unificación de valores en columnas "WindGustDir", "WindGustDir9am" y "WindGustDir3pm"**

In [ ]:
# Unificar valores columnas WindGustDir WindGustDir9am WindGustDir3pm

def conditions_dirs(column_name):
  conditions_dirs = [
        (dataset[column_name].isin(['N'])), # N
        (dataset[column_name].isin(['S'])), # S
        (dataset[column_name].isin(['E'])), # E
        (dataset[column_name].isin(['W'])), # W
        (dataset[column_name].isin(['NE', 'NNE', 'ENE'])), # NE
        (dataset[column_name].isin(['NW', 'WNW', 'NNW'])), # NW
        (dataset[column_name].isin(['SE', 'ESE', 'SSE'])), # SE
        (dataset[column_name].isin(['SW', 'SSW', 'WSW'])) # SW
    ]
  return conditions_dirs

dirs = ['N', 'S', 'E', 'W', 'NE', 'NW', 'SE', 'SW']

# nueva columna WindGustDir
dataset['WindGustDir'] = np.select(conditions_dirs('WindGustDir'), dirs)
dataset['WindDir9am'] = np.select(conditions_dirs('WindDir9am'), dirs)
dataset['WindDir3pm'] = np.select(conditions_dirs('WindDir3pm'), dirs)

dataset['WindGustDir'].unique()
dataset['WindDir9am'].unique()
dataset['WindDir3pm'].unique()

In [ ]:
dataset.sample(5)

In [ ]:
# eliminar columna Date y Location
dataset = dataset.drop(['Date'], axis=1)
dataset = dataset.drop(['Location'], axis=1)

#**ALGORITMOS**

Algoritmos a evaluar:

- Regresión logística
- Árboles de decisión 
- Random Forest
- Gradient Boosting 

**CONJUNTO DE DATOS (60% train, 20% test, 20% validation)**

In [ ]:
dataset['RainToday'] = dataset['RainToday'].map({"No":0, "Yes":1})
dataset['RainTomorrow'] = dataset['RainTomorrow'].map({"No":0, "Yes":1})

In [ ]:
# 60% train, 20% test, 20% validation
train, not_train = train_test_split(dataset, test_size=0.4, random_state=40)
validation, test = train_test_split(not_train, test_size=0.5, random_state=40)

train.shape, validation.shape, test.shape

In [ ]:
dataset.sample(5)

**DATAFRAME MAPPER**

In [ ]:
mapper = DataFrameMapper([
    (['MinTemp'], [MinMaxScaler()]),
    (['MaxTemp'], [MinMaxScaler()]),
    (['Rainfall'], [MinMaxScaler()]),
    (['Evaporation'], [MinMaxScaler()]),
    (['Sunshine'], [MinMaxScaler()]),
    (['WindGustDir'], [OneHotEncoder()]),
    (['WindDir9am'], [OneHotEncoder()]),
    (['WindDir3pm'], [OneHotEncoder()]),
    (['WindGustSpeed'], [MinMaxScaler()]),
    (['WindSpeed9am'], [MinMaxScaler()]),
    (['WindSpeed3pm'], [MinMaxScaler()]),
    (['Humidity9am'], [MinMaxScaler()]),
    (['Humidity3pm'], [MinMaxScaler()]),
    (['Pressure9am'], [MinMaxScaler()]),
    (['Pressure3pm'], [MinMaxScaler()]),
    (['Cloud9am'], [MinMaxScaler()]),
    (['Cloud3pm'], [MinMaxScaler()]),
    (['Temp9am'], [MinMaxScaler()]),
    (['Temp3pm'], [MinMaxScaler()]),
    (['RainToday'], None),
    (['Season'], [OneHotEncoder()]),
    (['Year'], [MinMaxScaler()]),
    (['Month'], [MinMaxScaler()]),
    (['Day'], [MinMaxScaler()]),
    (['Region'], [OneHotEncoder()]),
])
sample = train.sample(5, random_state=42)
sample

In [ ]:
mapper.fit(train)
# Sample transformado con el mapper completo
mapper.transform(sample)

**Función para evaluar modelos**

In [ ]:
def evaluate_model(model, set_names=('train', 'validation'), title='', show_cm=True):
    if title:
        display(title)
        
    final_metrics = defaultdict(list)
    
    if show_cm:
        fig, axis = plt.subplots(1, len(set_names), sharey=True, figsize=(15, 3))
    
    for i, set_name in enumerate(set_names):
        assert set_name in ['train', 'validation', 'test']
        set_data = globals()[set_name] 

        y = set_data.RainTomorrow
        y_pred = model.predict(set_data)
        final_metrics['Accuracy'].append(metrics.accuracy_score(y, y_pred))
        final_metrics['Precision'].append(metrics.precision_score(y, y_pred))
        
        if show_cm:
            ax = axis[i]
            sns.heatmap(metrics.confusion_matrix(y, y_pred), ax=ax, cmap='Blues', annot=True, fmt='.0f', cbar=False)

            ax.set_title(set_name)
            ax.xaxis.set_ticklabels(['No', 'Yes'])
            ax.yaxis.set_ticklabels(['No', 'Yes'])
            ax.set_xlabel('Predicted class (RainTomorrow)')
            ax.set_ylabel('True class (RainTomorrow)')

        
    display(pd.DataFrame(final_metrics, index=set_names))
    if show_cm:
        plt.tight_layout()
        plt.show()

# **REGRESIÓN LOGÍSTICA**

In [ ]:
lr_model = Pipeline([
    ('mapper', mapper),
    ('classifier', LogisticRegression(random_state=40, max_iter=1000)),
])
lr_model.fit(train, train.RainTomorrow)

y_pred = lr_model.predict(validation)
y_pred

In [ ]:
metrics.accuracy_score(validation.RainTomorrow, y_pred)

In [ ]:
lr_model2 = Pipeline([
    ('mapper', mapper),
    ('classifier', LogisticRegression(random_state=40, max_iter=10)),
])
lr_model2.fit(train, train.RainTomorrow)
evaluate_model(lr_model2)

In [ ]:
evaluate_model(lr_model)

In [ ]:
lr_model3 = Pipeline([
    ('mapper', mapper),
    ('classifier', LogisticRegression(random_state=40, solver='liblinear')),
])
lr_model3.fit(train, train.RainTomorrow)
evaluate_model(lr_model3)

**Análisis de métricas**

Como se muestra en las tres celdas anteriores, evaluamos el modelo cambiando ciertos parámetros (max_iter y solver). En todos los casos obtuvimos resultados muy similares que interpretamos a continuación.



*   En cuanto a Accuracy, nos indica que el modelo acertó aproximadamente el 85% de los casos que predijo.
*   Si vemos Precision, nos indica que de los casos en los que lloverá al día siguiente (RainTomorrow = Yes), encontró el 73% de ellos.



In [ ]:
clf = lr_model.named_steps['classifier']
clf.coef_  # los pesos que aprendió la regresión logística
df_importance = pd.DataFrame({
    'feature': mapper.transformed_names_, 
    'coeficiente': clf.coef_[0],
}).sort_values(by='coeficiente')

df = px.data.tips()
fig = px.bar(df_importance, x='coeficiente', y='feature', orientation='h',
             height=1500,
             color = "coeficiente",
             title='Coeficientes')
fig.show()

**Análisis de gráfica de coeficientes**

Los coeficientes (o parámetros) de la función de salida nos dan "pistas" para interpretar la influencia de los mismos en la predicción.
En este caso, vemos que los valores que tomen las variables WindGustSpeed, Humidity3pm y Pressure9am, mientras mayores sean, más probabilidad habrá de que al día siguiente llueva (esto se observa en la parte superior del gráfico anterior).

Por otro lado, si observamos la parte inferior, podemos decir que mientras mayores sean los valores de Pressure3pm y Sunshine, menor será la probabilidad de que llueva al día siguiente (debido a que son parámetros con signo negativo).



# **ÁRBOLES DE DECISIÓN** 

In [ ]:
tree_model1 = DecisionTreeClassifier(random_state=40)

dt_model1 = Pipeline([
                     ('mapper', mapper),
                     ('classifier', tree_model1),
])

history = dt_model1.fit(train, train.RainTomorrow)

evaluate_model(dt_model1, title='Decision Tree')

In [ ]:
tree_model = DecisionTreeClassifier(max_depth=3, random_state=40)

dt_model = Pipeline([
                     ('mapper', mapper),
                     ('classifier', tree_model),
])

history = dt_model.fit(train, train.RainTomorrow)

evaluate_model(dt_model, title='Decision Tree')

**Análisis de métricas**

Como se muestra en las celdas anteriores, se evaluó el algoritmo de Árboles de decisión de dos formas: sin establecer una profundidad máxima y estableciéndola. En el primer caso se observa que con el set de train sobreentenó debido a que tanto Accuracy como Precision devolvieron un valor de 1. En el set de validation esto no ocurrió.

Por otro lado, habiendo delimitado la profundidad máxima con un valor de 3, se obtuvieron métricas similares tanto para train como para validation. Accuracy nos indica que el modelo acertó en aproximadamente un 84% de los casos. Precision demuestra que el modelo encontró aproximadamente el 75% de los casos en los que al día siguiente lloverá.

In [ ]:
# Gráfico del árbol
def graph_tree(tree, col_names):
    graph_data = export_graphviz(
        tree, 
        out_file=None, 
        feature_names=col_names,  
        class_names=['died', 'survived'],  
        filled=True, 
        rounded=True,  
        special_characters=True,
    )
    graph = graphviz.Source(graph_data)  
    return graph

In [ ]:
graph_tree(tree_model, mapper.transformed_names_)

# **RANDOM FOREST**

In [ ]:
forest_model = RandomForestClassifier(max_depth=3, random_state=40)

rf_model = Pipeline([
    ('mapper', mapper),
    ('classifier', forest_model),
])

rf_model.fit(train, train.RainTomorrow)

evaluate_model(rf_model, title='Random Forest')

In [ ]:
forest_model = RandomForestClassifier(n_estimators=150, max_features=5, max_depth=3, random_state=40)

rf_model = Pipeline([
    ('mapper', mapper),
    ('classifier', forest_model),
])

rf_model.fit(train, train.RainTomorrow)

evaluate_model(rf_model, title='Random Forest')

In [ ]:
graph_tree(forest_model.estimators_[0], mapper.transformed_names_)

**Análisis de métricas**

Con random forest probamos dos formas, una con el parámetro max_depth, y otra sumandole los parametros n_estimators y max_features. En el primero podemos ver que acertó con un 82% de todos los casos con accuracy y en precision, de los casos en los que sí iba a llover, acertó un 86%.

En el segundo, se puede ver que con accuracy acertó el 81% de los casos y un 90% con precision.
Es decir, ajustando los parámetros n_estimators y max_features logramos que mejore en precision, pero afecta apenas negativamente en accuracy.

Nos damos cuenta que no está sobreentrenando porque, a comparación del modelo anterior, la métrica tanto en train como en validation tienen valores similares y nunca nos arrojó como resultado 1.

# **GRADIENT BOOSTING**

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
boo_model = GradientBoostingClassifier(random_state=40)
boost_model = Pipeline([
    ('mapper', mapper),
    ('classifier', boo_model),
])

boost_model.fit(train, train.RainTomorrow)
evaluate_model(boost_model, title='Gradient Boosting')

In [ ]:
boo_model = GradientBoostingClassifier(n_estimators=100, max_depth=3, max_features=2, random_state=40)
boost_model = Pipeline([
    ('mapper', mapper),
    ('classifier', boo_model),
])

boost_model.fit(train, train.RainTomorrow)
evaluate_model(boost_model, title='Gradient Boosting')

**Análisis de métricas**

Se realizó la evaluación del modelo con Gradient Boosting de dos maneras: sin limitar la cantidad de estimadores, profundidad de árboles y cantidad de features; y haciéndolo. Se puede observar que los resultados obtenidos de las métricas son muy similares, y que, para ambos casos, se puede decir que el modelo no esta sobreentrenando ya que los valores obtenidos tanto en test como en validation son cercanos.

Los resultados concluídos son:
- Accuracy nos indica que el modelo acertó el aproximadamente 86% de los casos que predijo.
- Precission nos indica que de los casos en los que lloverá al día siguiente (RainTomorrow = Yes), encontró el 75% de ellos.

**Gráfico del árbol**

In [ ]:
graph_tree(boo_model.estimators_[1][0], col_names=mapper.transformed_names_)

# **VALOR FINAL DE MÉTRICA A INFORMAR AL CLIENTE**

Evaluamos los modelos nuevamente, ahora teniendo en cuenta el dataset de test.

In [ ]:
evaluate_model(lr_model, title='Logistic Regression', set_names=('train', 'test', 'validation'), show_cm=False)
evaluate_model(dt_model, title='Árboles de decisión', set_names=('train', 'test', 'validation'), show_cm=False)
evaluate_model(rf_model, title='Random Forest', set_names=('train', 'test', 'validation'), show_cm=False)
evaluate_model(boost_model, title='Gradient Boosting', set_names=('train', 'test', 'validation'), show_cm=False)

Habiendo evaluado el set de test vemos que todos los modelos arrojan valores similares. Aún así, consideramos elegir la métrica Accuracy porque el equivocarse en falsos positivos no tiene un alto impacto, a menos que el cliente le de importancia por alguna situación particular, por la cual le daríamos el porcentaje de Precision. 
Con respecto al modelo, elegimos Gradient Boosting ya que nos parece el más completo en tener en cuenta las fallas que se van cometiendo en cada árbol para aprender de estos errores. Además, a comparación de los demás modelos, posee uno de los porcentajes más altos de acierto. Por lo tanto, la métrica final a informar al cliente sería del 85%.